In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2


%matplotlib inline

print("OK\n")

OK



## 共通関数定義

In [3]:
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
"""

params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 10000
}




def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=1000
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## インスタンス生成

In [4]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

## 特徴量生成

In [5]:
### application
df = FeatureEngineering2.application()
df.head()

"Application_Train_Test" final shape: (356250, 234)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,APP_EXT_SOURCE_2*DAYS_EMPLOYED,APP_EXT_SOURCE_3*DAYS_EMPLOYED,APPS_GOODS_INCOME_RATIO,APPS_CNT_FAM_INCOME_RATIO,APPS_INCOME_EMPLOYED_RATIO,CREDIT_TO_GOODS_RATIO_2,APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio,APP_INCOME_TO_EMPLOYED_RATIO,APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,-167.498254,-88.782372,1.733333,202500.0,-317.896389,1.158397,-7825.5,-317.896389,1.780220,8824.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,-739.227981,NaN,4.183333,135000.0,-227.272727,1.145199,-13198.5,-227.272727,0.696970,15577.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,-125.080219,-164.152505,2.000000,67500.0,-300.000000,1.000000,-1125.0,-300.000000,3.622222,18821.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,-1976.692297,NaN,2.200000,67500.0,-44.422507,1.052803,-18436.5,-44.422507,0.203027,15966.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,-980.478916,NaN,4.222222,121500.0,-39.993417,1.000000,-11740.5,-39.993417,0.364055,16894.0


In [6]:
### bureau_balance
df = df.merge(FeatureEngineering2.bureau_bb(), how='left', on='SK_ID_CURR')
df.head()

"Bureau/Bureau Balance" final shape: (305811, 200)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MIN,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MAX,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MEAN,CLOSED_DEBT_CREDIT_DIFF_MIN,CLOSED_DEBT_CREDIT_DIFF_MAX,CLOSED_DEBT_CREDIT_DIFF_MEAN,CLOSED_BUREAU_IS_DPD_MEAN,CLOSED_BUREAU_IS_DPD_SUM,CLOSED_BUREAU_IS_DPD_OVER120_MEAN,CLOSED_BUREAU_IS_DPD_OVER120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,135000.0,85245.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,22248.0,112500.0,69133.5,0.0,0.0,0.0,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,94500.0,94537.8,94518.9,0.0,0.0,0.0,0.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,146250.0,146250.0,146250.0,0.0,0.0,0.0,0.0


In [7]:
### previos_application
df = df.merge(FeatureEngineering2.previous_application(), how='left', on='SK_ID_CURR')
df.head()

"Previous Applications" final shape: (338857, 321)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REFUSED_PREV_GOODS_DIFF_MEAN,REFUSED_PREV_GOODS_DIFF_MAX,REFUSED_PREV_GOODS_DIFF_SUM,REFUSED_PREV_GOODS_APPL_RATIO_MEAN,REFUSED_PREV_GOODS_APPL_RATIO_MAX,REFUSED_DAYS_LAST_DUE_DIFF_MEAN,REFUSED_DAYS_LAST_DUE_DIFF_MAX,REFUSED_DAYS_LAST_DUE_DIFF_SUM,REFUSED_SIMPLE_INTERESTS_MEAN,REFUSED_SIMPLE_INTERESTS_MAX
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,1.0,1.0,NaN,NaN,0.0,0.015231,0.015231
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
### pos_cash
df = df.merge(FeatureEngineering2.pos_cash(), how='left', on='SK_ID_CURR')
df.head()

"Pos-Cash" balance final shape: (337252, 46)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT,POS_LOAN_COMPLETED_MEAN,POS_COMPLETED_BEFORE_MEAN,POS_REMAINING_INSTALMENTS,POS_REMAINING_INSTALMENTS_RATIO,LATE_PAYMENT_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.000000,0.000000,0.0,0.0,19.0,0.000000,0.0,6.0,0.250000,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.000000,0.000000,0.0,0.0,28.0,0.250000,1.0,1.0,0.083333,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.000000,0.000000,0.0,0.0,4.0,0.250000,1.0,0.0,0.000000,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.047619,0.000000,0.0,0.0,21.0,0.458333,2.0,3.0,0.250000,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.000000,0.015152,0.0,0.0,66.0,0.209402,1.0,14.0,0.641667,0.0


In [9]:
### instllment_payment
df = df.merge(FeatureEngineering2.installment(), how='left', on='SK_ID_CURR')
df.head()

"Installments Payments" final shape: (339587, 85)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_D365PAYMENT_PERC_MAX,INS_D365DPD_DIFF_MEAN,INS_D365DPD_DIFF_MIN,INS_D365DPD_DIFF_MAX,INS_D365DPD_MEAN,INS_D365DPD_SUM,INS_D365INS_IS_DPD_UNDER_120_MEAN,INS_D365INS_IS_DPD_UNDER_120_SUM,INS_D365INS_IS_DPD_OVER_120_MEAN,INS_D365INS_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1.0,-17.363636,-24.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,1.0,-6.181818,-23.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,1.0,-2.250000,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
### credit_card_balance
df = df.merge(FeatureEngineering2.credit_card(), how='left', on='SK_ID_CURR')
df.head()

"Credit Card Balance" final shape: (103558, 284)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_48M_SK_DPD_MAX,INS_48M_SK_DPD_SUM,INS_48M_DRAWING_LIMIT_RATIO_MIN,INS_48M_DRAWING_LIMIT_RATIO_MAX,INS_48M_LATE_PAYMENT_MEAN,INS_48M_LATE_PAYMENT_SUM,INS_48M_CARD_IS_DPD_UNDER_120_MEAN,INS_48M_CARD_IS_DPD_UNDER_120_SUM,INS_48M_CARD_IS_DPD_OVER_120_MEAN,INS_48M_CARD_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
### 変数選択001
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1168 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1122 features are remained after removing non-informative features
123 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 999 features are remained after removing features not interesting for LightGBM classifier
---=> 58 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1057 features

---=> df final shape: (356250, 1057)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,0,0,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,0,0,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,0,0,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,1,1,0,0,0,1,0,1,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,0,0,0,0,0,0,0


In [12]:
### 変数選択002
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1057 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1051 features are remained after removing non-informative features
40 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 1011 features are remained after removing features not interesting for LightGBM classifier
---=> 19 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1030 features

---=> df final shape: (356250, 1030)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,1,0,0,1,1,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0,1,1,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,1,0,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,1,1,0,0,0,0,0


In [13]:
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,1,0,0,1,1,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0,1,1,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,1,0,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,1,1,0,0,0,0,0


## 学習

### 001 baseline

In [14]:
# split train, and test datasets
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

(48744, 1030)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
307506,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,0,0,1,1,0,0,0,0,0
307507,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,0,0,1,1,0,0,0,0,0
307508,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,0,0,1,1,0,1,0,1,0
307509,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0,1,1,0,1,0,1,0
307510,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,1,0,0,0,0,0


In [15]:
train_setB1a = pd.read_csv('./home-credit-default-risk/exports/train_setB1a.csv')
test_setB1a =  pd.read_csv('./home-credit-default-risk/exports/test_setB1a.csv')
test_setB1a.head()

,SK_ID_CURR,setB1a
0,100001,0.033433
1,100005,0.098996
2,100013,0.042185
3,100028,0.039429
4,100038,0.161166


In [16]:
app_train = app_train.merge(train_setB1a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_setB1a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,1,1,0,0,0,0,0,0.033433
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,1,1,0,0,0,0,0,0.098996
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,0,1,1,0,1,0,1,0,0.042185
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,0,1,1,0,1,0,1,0,0.039429
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,1,0,0,0,0,0,0.161166


In [17]:
train_setB1xgb = pd.read_csv('./home-credit-default-risk/exports/train_setB1xgb.csv')
test_setB1xgb =  pd.read_csv('./home-credit-default-risk/exports/test_setB1xgb.csv')
test_setB1xgb.head()

,SK_ID_CURR,setB1x
0,100001,0.032762
1,100005,0.100620
2,100013,0.044999
3,100028,0.034211
4,100038,0.172532


In [18]:
app_train = app_train.merge(train_setB1xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_setB1xgb, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,0,0,0,0,0.033433,0.032762
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,1,0,0,0,0,0,0.098996,0.100620
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,1,0,1,0,1,0,0.042185,0.044999
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,1,0,1,0,1,0,0.039429,0.034211
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,0,0,0,0,0.161166,0.172532


In [19]:
pred_train_set1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_set1a.csv')
pred_test_set1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_set1a.csv')
pred_test_set1a.head()

,SK_ID_CURR,set1
0,100001,0.040068
1,100005,0.095240
2,100013,0.044435
3,100028,0.036586
4,100038,0.141754


In [20]:
app_train = app_train.merge(pred_train_set1a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(pred_test_set1a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,1,0,0,0,0,0,0.033433,0.032762,0.040068
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,1,0,0,0,0,0,0.098996,0.100620,0.095240
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,1,0,1,0,1,0,0.042185,0.044999,0.044435
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,1,0,1,0,1,0,0.039429,0.034211,0.036586
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,1,0,0,0,0,0,0.161166,0.172532,0.141754


In [21]:
train_set1xgb = pd.read_csv('./home-credit-default-risk/exports/train_set1xgb.csv')
test_set1xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set1xgb.csv')
test_set1xgb.head()

,SK_ID_CURR,set1x
0,100001,0.035234
1,100005,0.107426
2,100013,0.048029
3,100028,0.032975
4,100038,0.161962


In [22]:
app_train = app_train.merge(train_set1xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set1xgb, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,0,0,0,0,0,0.033433,0.032762,0.040068,0.035234
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,0,0,0,0,0,0.098996,0.100620,0.095240,0.107426
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,1,0,1,0,0.042185,0.044999,0.044435,0.048029
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,1,0,1,0,0.039429,0.034211,0.036586,0.032975
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,1,0,0,0,0,0,0.161166,0.172532,0.141754,0.161962


In [23]:
train_set2xgb = pd.read_csv('./home-credit-default-risk/exports/train_set2xgb.csv')
test_set2xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set2xgb.csv')
test_set2xgb.head()

,SK_ID_CURR,set2x
0,100001,0.035013
1,100005,0.103532
2,100013,0.048415
3,100028,0.032249
4,100038,0.150442


In [24]:
app_train = app_train.merge(train_set2xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set2xgb, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0,0,0,0.033433,0.032762,0.040068,0.035234,0.035013
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0,0,0,0.098996,0.100620,0.095240,0.107426,0.103532
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,0,0.042185,0.044999,0.044435,0.048029,0.048415
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,0,0.039429,0.034211,0.036586,0.032975,0.032249
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,0,0.161166,0.172532,0.141754,0.161962,0.150442


In [25]:
train_set2nn = pd.read_csv('./home-credit-default-risk/exports/train_set2nn.csv')
test_set2nn =  pd.read_csv('./home-credit-default-risk/exports/test_set2nn.csv')
test_set2nn.head()

,SK_ID_CURR,set2nn
0,100001,0.048972
1,100005,0.141351
2,100013,0.028088
3,100028,0.039056
4,100038,0.192637


In [26]:
app_train = app_train.merge(train_set2nn, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set2nn, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2x,set2nn
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0,0,0.033433,0.032762,0.040068,0.035234,0.035013,0.048972
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0,0,0.098996,0.100620,0.095240,0.107426,0.103532,0.141351
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,1,0,0.042185,0.044999,0.044435,0.048029,0.048415,0.028088
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,1,0,0.039429,0.034211,0.036586,0.032975,0.032249,0.039056
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,0.161166,0.172532,0.141754,0.161962,0.150442,0.192637


In [27]:
train_set3a = pd.read_csv('./home-credit-default-risk/exports/train_set3a.csv')
test_set3a =  pd.read_csv('./home-credit-default-risk/exports/test_set3a.csv')
test_set3a.head()

,SK_ID_CURR,set3a
0,100001,0.049120
1,100005,0.115729
2,100013,0.064166
3,100028,0.038931
4,100038,0.195575


In [28]:
app_train = app_train.merge(train_set3a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set3a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2x,set2nn,set3a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0,0.033433,0.032762,0.040068,0.035234,0.035013,0.048972,0.049120
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0,0.098996,0.100620,0.095240,0.107426,0.103532,0.141351,0.115729
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,0.042185,0.044999,0.044435,0.048029,0.048415,0.028088,0.064166
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,0.039429,0.034211,0.036586,0.032975,0.032249,0.039056,0.038931
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0.161166,0.172532,0.141754,0.161962,0.150442,0.192637,0.195575


In [29]:
train_set3xgb = pd.read_csv('./home-credit-default-risk/exports/train_set3xgb.csv')
test_set3xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set3xgb.csv')
test_set3xgb.head()

,SK_ID_CURR,set3x
0,100001,0.034310
1,100005,0.127556
2,100013,0.044872
3,100028,0.030915
4,100038,0.201295


In [30]:
app_train = app_train.merge(train_set3xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set3xgb, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2x,set2nn,set3a,set3x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0.033433,0.032762,0.040068,0.035234,0.035013,0.048972,0.049120,0.034310
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0.098996,0.100620,0.095240,0.107426,0.103532,0.141351,0.115729,0.127556
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,0.042185,0.044999,0.044435,0.048029,0.048415,0.028088,0.064166,0.044872
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0.039429,0.034211,0.036586,0.032975,0.032249,0.039056,0.038931,0.030915
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0.161166,0.172532,0.141754,0.161962,0.150442,0.192637,0.195575,0.201295


In [31]:
train_set4a = pd.read_csv('./home-credit-default-risk/exports/train_set4a.csv')
test_set4a =  pd.read_csv('./home-credit-default-risk/exports/test_set4a.csv')
test_set4a.head()

,SK_ID_CURR,set4a
0,100001,0.042113
1,100005,0.132192
2,100013,0.057271
3,100028,0.042206
4,100038,0.195048


In [32]:
app_train = app_train.merge(train_set4a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set4a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2x,set2nn,set3a,set3x,set4a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0.033433,0.032762,0.040068,0.035234,0.035013,0.048972,0.049120,0.034310,0.042113
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0.098996,0.100620,0.095240,0.107426,0.103532,0.141351,0.115729,0.127556,0.132192
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,0.042185,0.044999,0.044435,0.048029,0.048415,0.028088,0.064166,0.044872,0.057271
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,0.039429,0.034211,0.036586,0.032975,0.032249,0.039056,0.038931,0.030915,0.042206
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0.161166,0.172532,0.141754,0.161962,0.150442,0.192637,0.195575,0.201295,0.195048


In [33]:
train_set4xgb = pd.read_csv('./home-credit-default-risk/exports/train_set4xgb.csv')
test_set4xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set4xgb.csv')
test_set4xgb.head()

,SK_ID_CURR,set4x
0,100001,0.034978
1,100005,0.118268
2,100013,0.039774
3,100028,0.037283
4,100038,0.164890


In [34]:
app_train = app_train.merge(train_set4xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set4xgb, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,setB1a,setB1x,set1,set1x,set2x,set2nn,set3a,set3x,set4a,set4x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0.033433,0.032762,0.040068,0.035234,0.035013,0.048972,0.049120,0.034310,0.042113,0.034978
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0.098996,0.100620,0.095240,0.107426,0.103532,0.141351,0.115729,0.127556,0.132192,0.118268
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0.042185,0.044999,0.044435,0.048029,0.048415,0.028088,0.064166,0.044872,0.057271,0.039774
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0.039429,0.034211,0.036586,0.032975,0.032249,0.039056,0.038931,0.030915,0.042206,0.037283
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0.161166,0.172532,0.141754,0.161962,0.150442,0.192637,0.195575,0.201295,0.195048,0.164890


In [35]:
train_set5a = pd.read_csv('./home-credit-default-risk/exports/train_set5a.csv')
test_set5a =  pd.read_csv('./home-credit-default-risk/exports/test_set5a.csv')
test_set4a.head()

,SK_ID_CURR,set4a
0,100001,0.042113
1,100005,0.132192
2,100013,0.057271
3,100028,0.042206
4,100038,0.195048


In [36]:
app_train = app_train.merge(train_set5a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set5a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,setB1x,set1,set1x,set2x,set2nn,set3a,set3x,set4a,set4x,set5a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0.032762,0.040068,0.035234,0.035013,0.048972,0.049120,0.034310,0.042113,0.034978,0.034370
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0.100620,0.095240,0.107426,0.103532,0.141351,0.115729,0.127556,0.132192,0.118268,0.116919
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0.044999,0.044435,0.048029,0.048415,0.028088,0.064166,0.044872,0.057271,0.039774,0.037191
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0.034211,0.036586,0.032975,0.032249,0.039056,0.038931,0.030915,0.042206,0.037283,0.051958
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0.172532,0.141754,0.161962,0.150442,0.192637,0.195575,0.201295,0.195048,0.164890,0.141608


In [37]:
train_set5xgb = pd.read_csv('./home-credit-default-risk/exports/train_set5xgb.csv')
test_set5xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set5xgb.csv')
test_set5xgb.head()

,SK_ID_CURR,set5x
0,100001,0.028969
1,100005,0.116288
2,100013,0.041057
3,100028,0.045375
4,100038,0.136362


In [38]:
app_train = app_train.merge(train_set5xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set5xgb, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,set1,set1x,set2x,set2nn,set3a,set3x,set4a,set4x,set5a,set5x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0.040068,0.035234,0.035013,0.048972,0.049120,0.034310,0.042113,0.034978,0.034370,0.028969
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0.095240,0.107426,0.103532,0.141351,0.115729,0.127556,0.132192,0.118268,0.116919,0.116288
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0.044435,0.048029,0.048415,0.028088,0.064166,0.044872,0.057271,0.039774,0.037191,0.041057
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0.036586,0.032975,0.032249,0.039056,0.038931,0.030915,0.042206,0.037283,0.051958,0.045375
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0.141754,0.161962,0.150442,0.192637,0.195575,0.201295,0.195048,0.164890,0.141608,0.136362


In [39]:
### indexを削除
app_train=app_train.reset_index()
app_test=app_test.reset_index()
app_test.head()

,level_0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,set1,set1x,set2x,set2nn,set3a,set3x,set4a,set4x,set5a,set5x
0,0,0,100001,NaN,1,0,0,0,135000.0,568800.0,...,0.040068,0.035234,0.035013,0.048972,0.049120,0.034310,0.042113,0.034978,0.034370,0.028969
1,1,1,100005,NaN,0,0,0,0,99000.0,222768.0,...,0.095240,0.107426,0.103532,0.141351,0.115729,0.127556,0.132192,0.118268,0.116919,0.116288
2,2,2,100013,NaN,0,1,0,0,202500.0,663264.0,...,0.044435,0.048029,0.048415,0.028088,0.064166,0.044872,0.057271,0.039774,0.037191,0.041057
3,3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,...,0.036586,0.032975,0.032249,0.039056,0.038931,0.030915,0.042206,0.037283,0.051958,0.045375
4,4,4,100038,NaN,0,1,1,1,180000.0,625500.0,...,0.141754,0.161962,0.150442,0.192637,0.195575,0.201295,0.195048,0.164890,0.141608,0.136362


In [40]:
### indexを削除
app_train = app_train.drop(columns=['index'])
app_test = app_test.drop(columns=['index'])
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,set1,set1x,set2x,set2nn,set3a,set3x,set4a,set4x,set5a,set5x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0.040068,0.035234,0.035013,0.048972,0.049120,0.034310,0.042113,0.034978,0.034370,0.028969
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0.095240,0.107426,0.103532,0.141351,0.115729,0.127556,0.132192,0.118268,0.116919,0.116288
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0.044435,0.048029,0.048415,0.028088,0.064166,0.044872,0.057271,0.039774,0.037191,0.041057
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0.036586,0.032975,0.032249,0.039056,0.038931,0.030915,0.042206,0.037283,0.051958,0.045375
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0.141754,0.161962,0.150442,0.192637,0.195575,0.201295,0.195048,0.164890,0.141608,0.136362


### 正規化＋対数変換

In [41]:
app_train_ord = app_train.copy()
app_test_ord = app_test.copy()
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,set1,set1x,set2x,set2nn,set3a,set3x,set4a,set4x,set5a,set5x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0.040068,0.035234,0.035013,0.048972,0.049120,0.034310,0.042113,0.034978,0.034370,0.028969
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0.095240,0.107426,0.103532,0.141351,0.115729,0.127556,0.132192,0.118268,0.116919,0.116288
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0.044435,0.048029,0.048415,0.028088,0.064166,0.044872,0.057271,0.039774,0.037191,0.041057
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0.036586,0.032975,0.032249,0.039056,0.038931,0.030915,0.042206,0.037283,0.051958,0.045375
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0.141754,0.161962,0.150442,0.192637,0.195575,0.201295,0.195048,0.164890,0.141608,0.136362


In [42]:
# infをnanへ置換
app_train_ord = app_train_ord.replace([np.inf, -np.inf], np.nan)
app_test_ord = app_test_ord.replace([np.inf, -np.inf], np.nan)
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,set1,set1x,set2x,set2nn,set3a,set3x,set4a,set4x,set5a,set5x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0.040068,0.035234,0.035013,0.048972,0.049120,0.034310,0.042113,0.034978,0.034370,0.028969
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0.095240,0.107426,0.103532,0.141351,0.115729,0.127556,0.132192,0.118268,0.116919,0.116288
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0.044435,0.048029,0.048415,0.028088,0.064166,0.044872,0.057271,0.039774,0.037191,0.041057
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0.036586,0.032975,0.032249,0.039056,0.038931,0.030915,0.042206,0.037283,0.051958,0.045375
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0.141754,0.161962,0.150442,0.192637,0.195575,0.201295,0.195048,0.164890,0.141608,0.136362


In [43]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(app_test_ord.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [44]:
### 欠損値補完
#app_train_ord = app_train_ord.fillna(app_train_ord.max() + (app_train_ord.max() * 0.4))
app_train_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,set1,set1x,set2x,set2nn,set3a,set3x,set4a,set4x,set5a,set5x
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.260928,0.199398,0.198794,0.240473,0.223662,0.259405,0.218256,0.207998,0.196429,0.244814
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.012171,0.012061,0.012648,0.014736,0.024137,0.023080,0.012283,0.013902,0.013261,0.014884
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.039950,0.047440,0.033060,0.029793,0.035314,0.033290,0.033160,0.034973,0.030715,0.036912
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.030776,0.030071,0.025442,0.031804,0.031135,0.022860,0.029616,0.040586,0.034176,0.037270
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.035088,0.023555,0.039186,0.060946,0.035167,0.033131,0.035875,0.036123,0.038256,0.035143


In [45]:
### 欠損値補完
#app_test_ord = app_test_ord.fillna(app_test_ord.max() + (app_test_ord.max() * 0.4))
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,set1,set1x,set2x,set2nn,set3a,set3x,set4a,set4x,set5a,set5x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0.040068,0.035234,0.035013,0.048972,0.049120,0.034310,0.042113,0.034978,0.034370,0.028969
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0.095240,0.107426,0.103532,0.141351,0.115729,0.127556,0.132192,0.118268,0.116919,0.116288
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0.044435,0.048029,0.048415,0.028088,0.064166,0.044872,0.057271,0.039774,0.037191,0.041057
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0.036586,0.032975,0.032249,0.039056,0.038931,0.030915,0.042206,0.037283,0.051958,0.045375
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0.141754,0.161962,0.150442,0.192637,0.195575,0.201295,0.195048,0.164890,0.141608,0.136362


In [46]:
### Yao-Jhonson変換
#for column in encoding_columns:
#    modules.process_yj_convert(app_train_ord, app_test_ord, column, [column])#
#
#app_test_ord.head()

In [47]:
### indexを設定
app_train_ord = app_train_ord.set_index('SK_ID_CURR')
app_test_ord = app_test_ord.set_index('SK_ID_CURR')
app_test_ord.head()

,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,set1,set1x,set2x,set2nn,set3a,set3x,set4a,set4x,set5a,set5x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0.040068,0.035234,0.035013,0.048972,0.049120,0.034310,0.042113,0.034978,0.034370,0.028969
100005,1,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0.095240,0.107426,0.103532,0.141351,0.115729,0.127556,0.132192,0.118268,0.116919,0.116288
100013,2,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,0.044435,0.048029,0.048415,0.028088,0.064166,0.044872,0.057271,0.039774,0.037191,0.041057
100028,3,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,0.036586,0.032975,0.032249,0.039056,0.038931,0.030915,0.042206,0.037283,0.051958,0.045375
100038,4,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0.141754,0.161962,0.150442,0.192637,0.195575,0.201295,0.195048,0.164890,0.141608,0.136362


In [48]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(app_train_ord, app_test_ord, num_cols)
        
app_test_ord.head()

,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,set1,set1x,set2x,set2nn,set3a,set3x,set4a,set4x,set5a,set5x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1.732045,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.451791,-0.239271,...,-0.419358,-0.450892,-0.457814,-0.317197,-0.328307,-0.469542,-0.401764,-0.461862,-0.475811,-0.514803
100005,-1.732034,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.671920,-0.970093,...,0.159147,0.319855,0.261749,0.678170,0.365058,0.507163,0.544776,0.422332,0.374416,0.403633
100013,-1.732023,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.943903,0.247943,...,-0.373564,-0.314285,-0.317065,-0.542220,-0.171688,-0.358904,-0.242480,-0.410956,-0.446752,-0.387664
100028,-1.732011,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.511669,2.805819,...,-0.455868,-0.475006,-0.486834,-0.424040,-0.434369,-0.505095,-0.400781,-0.437400,-0.294660,-0.342247
100038,-1.732000,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.342100,0.235763,...,0.646859,0.902093,0.754380,1.230772,1.196224,1.279548,1.205262,0.917270,0.628698,0.614771


In [49]:
### indexを削除
app_train_ord = app_train_ord.reset_index()
app_test_ord = app_test_ord.reset_index()
app_test_ord.head()

,SK_ID_CURR,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,set1,set1x,set2x,set2nn,set3a,set3x,set4a,set4x,set5a,set5x
0,100001,-1.732045,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.451791,...,-0.419358,-0.450892,-0.457814,-0.317197,-0.328307,-0.469542,-0.401764,-0.461862,-0.475811,-0.514803
1,100005,-1.732034,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.671920,...,0.159147,0.319855,0.261749,0.678170,0.365058,0.507163,0.544776,0.422332,0.374416,0.403633
2,100013,-1.732023,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.943903,...,-0.373564,-0.314285,-0.317065,-0.542220,-0.171688,-0.358904,-0.242480,-0.410956,-0.446752,-0.387664
3,100028,-1.732011,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.511669,...,-0.455868,-0.475006,-0.486834,-0.424040,-0.434369,-0.505095,-0.400781,-0.437400,-0.294660,-0.342247
4,100038,-1.732000,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.342100,...,0.646859,0.902093,0.754380,1.230772,1.196224,1.279548,1.205262,0.917270,0.628698,0.614771


In [50]:
### level_0を削除
app_train_ord.drop('level_0', axis=1, inplace=True)
app_test_ord.drop('level_0', axis=1, inplace=True)
app_test_ord.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,set1,set1x,set2x,set2nn,set3a,set3x,set4a,set4x,set5a,set5x
0,100001,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.451791,-0.239271,...,-0.419358,-0.450892,-0.457814,-0.317197,-0.328307,-0.469542,-0.401764,-0.461862,-0.475811,-0.514803
1,100005,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.671920,-0.970093,...,0.159147,0.319855,0.261749,0.678170,0.365058,0.507163,0.544776,0.422332,0.374416,0.403633
2,100013,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.943903,0.247943,...,-0.373564,-0.314285,-0.317065,-0.542220,-0.171688,-0.358904,-0.242480,-0.410956,-0.446752,-0.387664
3,100028,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.511669,2.805819,...,-0.455868,-0.475006,-0.486834,-0.424040,-0.434369,-0.505095,-0.400781,-0.437400,-0.294660,-0.342247
4,100038,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.342100,0.235763,...,0.646859,0.902093,0.754380,1.230772,1.196224,1.279548,1.205262,0.917270,0.628698,0.614771


In [51]:
x_train = app_train_ord.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train_ord['TARGET']
id_train = app_train_ord[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [52]:
# データセット作成
x_test = app_test_ord.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test_ord[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [53]:
#del app_train_ord
#del app_test_ord
#gc.collect()

In [54]:
#for col in app_train_ord.columns:
#    print(col)

In [55]:
### パラメータ
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 3000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
"""
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 2000
}
"""
# Case3 2022/07/29
params = {
    #'boosting_type': 'goss'
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread':4
    ,'n_estimators':2000
    ,'learning_rate':0.005134
    ,'num_leaves':54
    ,'colsample_bytree':0.508716
    ,'subsample':1
    ,'max_depth':10
    ,'reg_alpha':0.436193
    ,'reg_lambda':0.479169
    ,'min_split_gain':0.024766
    ,'min_child_weight':40
    ,'silent':-1
    ,'verbose':-1
    ,'is_unbalance':False
}


### 学習モデル構築
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 1039) (61502, 1039)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=4 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.803246	valid_1's auc: 0.793873
[200]	training's auc: 0.805771	valid_1's auc: 0.793841
[300]	training's auc: 0.808398	valid_1's auc: 0.793776
[400]	training's auc: 0.811286	valid_1's auc: 0.793841
[500]	training's auc: 0.814142	valid_1's auc: 0.79388
[600]	training's auc: 0.817141	valid_1's auc: 0.793887
[700]	training's auc: 0.820301	valid_1's auc: 0.793818
[800]	training's auc: 0.823329	valid_1's auc: 0.793828
[900]	training's auc: 0.825907	valid_1's auc: 0.793773
[1000]	training's auc: 0.828349	valid_1's auc: 0.793695
[1100]	training's auc: 0.830501	valid_1's auc: 0.793633
[1200]	training's auc: 0.832823	valid_1's auc: 0.793582
[1300]	training's auc: 0.835074	valid_1's auc: 0.793587
[1400]	training's auc: 0.837419	valid_1's auc: 0.793581
[1500]	training's auc: 0.839643	valid_1'

In [56]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1027,set1,2132.2,78.953784
1030,set2x,2124.4,104.562422
1031,set3a,1597.4,41.776788
1034,set4x,1142.8,94.579067
1036,set5x,1071.8,90.932392
727,PAYMENT_RATE,709.2,102.485121
140,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff,647.8,41.984521
1028,set1x,583.8,84.402014
1035,set5a,581.0,61.947558
158,APP_SCORE2_TO_POP_RATIO,497.2,33.581245


In [57]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [58]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineeringOtherSource2-2.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.034323
1,100005,0.118494
2,100013,0.055381
3,100028,0.040814
4,100038,0.157623


## パラメータチューニング

In [59]:
#sampler = optuna.samplers.TPESampler(seed=123)
#study = optuna.create_study(sampler=sampler, direction='maximize')
#study.optimize(objective, n_trials=50, n_jobs=5)

In [60]:
#trial = study.best_trial
#print("auc(best)={:.4f}".format(trial.value))
#display(trial.params)

In [61]:
#params_best = trial.params
#params_best.update(params_base)
#display(params_best)

In [62]:
#train_oof, imp, metrics = baseline.train_lgb(x_train
#                                    ,y_train
#                                    ,id_train
#                                    ,list_nfold=[0,1,2,3,4]
#                                    ,n_splits=5
#                                    ,params=params_best)

In [63]:
#app_test_ord

In [64]:
### 推論用データセットの作成
#x_test = app_test_ord.drop(columns=['SK_ID_CURR','TARGET'])
#id_test = app_test_ord[['SK_ID_CURR']]

### カテゴリ変数をcategory型へ変換
#for col in x_test.columns:
#    if x_test[col].dtype=="O":
#        x_test[col] = x_test[col].astype('category')

### predict
#test_pred = baseline.predict_lgb(x_test
#                       ,id_test
#                       ,list_nfold=[0,1,2,3,4])

### make submission-file
#df_submit = test_pred.rename(columns={"pred":"TARGET"})
#print(df_submit.shape)
#display(df_submit.head())
#df_submit.to_csv("./home-credit-default-risk/submit/submission_HyperParameterTuningLgb.csv", index=None)

In [65]:
#del app_train_ord
#del app_test_ord
#gc.collect()